# Battle of the Neighbourhoods: The Best Place in London for Starting a Venue

## Introducing the Problem

London is a vast metropolitan area, with a large restaurant trade. The city has a population of around 8.9 million. 

My client wants to set up a business supplying restaurants with ingredients. In the first instance, they would like to develop their understanding of London’s restaurants and how they are distributed in the various buroughs. 

My task is to provide insights on the ways in which restaurant types are distributed in London buroughs.

## Data Requirements

To provide our initial recommendations, we want to provide a map of London that shows the geographical distribution of restaurants which specialise in food from the Indian and broader Indian subcontinent. Initially, we want to focus this analysis on a borough level. 

To do this, we will need: 

- London borough geo-spatial data.
- Data about the location of venues in London, which we can obtain from Foursquare API. 


### Step One: Preparation. First, let's import the packages.

In [400]:
# library for BeautifulSoup, for web scrapping
from bs4 import BeautifulSoup# library to handle data in a vectorized manner
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)# library to handle JSON files
import json
!pip -q install geopy
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!pip -q install geocoder
import geocoder# import time
import time
!pip -q install folium
import folium # map rendering library
import random 
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from IPython.display import display_html
print('...Done')


...Done


### Step Two: Let's Make a folium map of London

Let's make a map of London

In [401]:
LON_LATITUDE = '51.5074' 
LON_LONGITUDE = '0.1278' 
London_map = folium.Map(location = [LON_LATITUDE, LON_LONGITUDE], zoom_start = 11)
folium.Marker([LON_LATITUDE, LON_LONGITUDE]).add_to(London_map)
London_map

### Step Three: Get London Boroughs

In [402]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_London_boroughs').text
soup = BeautifulSoup(source, 'lxml')
soup.encode("utf-8-sig")
BoroughName = []
Population = []
Coordinates = []

for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if len(cells) > 0:
        BoroughName.append(cells[0].text.rstrip('\n'))
        Population.append(cells[7].text.rstrip('\n'))
        Coordinates.append(cells[8].text.rstrip('\n'))

In [403]:
dict = {'BoroughName' : BoroughName,
       'Population' : Population,
       'Coordinates': Coordinates}
London = pd.DataFrame.from_dict(dict)
London.head()


,BoroughName,Population,Coordinates
0,Barking and Dagenham [note 1],"194,352",51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ /...
1,Barnet,"369,088",51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W﻿ /...
2,Bexley,"236,687",51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E﻿ /...
3,Brent,"317,264",51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W﻿ /...
4,Bromley,"317,899",51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E﻿ /...


Now, we clean that data and create longitude and latitude columns from the coordinates columns. 

In [404]:
London['BoroughName'] = London['BoroughName'].map(lambda x: x.rstrip(']'))
London['BoroughName'] = London['BoroughName'].map(lambda x: x.rstrip('1234567890.'))
London['BoroughName'] = London['BoroughName'].str.replace('note','')
London['BoroughName'] = London['BoroughName'].map(lambda x: x.rstrip(' ['))
London[['Coordinates1','Coordinates2','Coordinates3']] = London['Coordinates'].str.split('/',expand=True)
London.drop(labels=['Coordinates','Coordinates1','Coordinates2'], axis=1,inplace = True)
London[['Latitude','Longitude']] = London['Coordinates3'].str.split(';',expand=True)
London.drop(labels=['Coordinates3'], axis=1,inplace = True)
London['Latitude'] = London['Latitude'].map(lambda x: x.rstrip(u'\ufeff'))
London['Latitude'] = London['Latitude'].map(lambda x: x.lstrip())
London['Longitude'] = London['Longitude'].map(lambda x: x.rstrip(')'))
London['Longitude'] = London['Longitude'].map(lambda x: x.rstrip('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ '))
London['Longitude'] = London['Longitude'].map(lambda x: x.rstrip(' ('))
London['Longitude'] = London['Longitude'].map(lambda x: x.rstrip(u'\ufeff'))
London['Longitude'] = London['Longitude'].map(lambda x: x.lstrip())
London['Population'] = London['Population'].str.replace(',','')
London.head()

,BoroughName,Population,Latitude,Longitude
0,Barking and Dagenham,194352,51.5607,0.1557
1,Barnet,369088,51.6252,-0.1517
2,Bexley,236687,51.4549,0.1505
3,Brent,317264,51.5588,-0.2817
4,Bromley,317899,51.4039,0.0198


### Step four: Get the Data about Indian Restaurants in London

In [405]:
FOURSQUARE_CLIENT_ID = '2JRZW0IGGPKHQ3BZBS0ZH0C14IG2SZMPVDDGPH2FR1QOZFNE'
FOURSQUARE_CLIENT_SECRET = 'PCTJNI53DJFN3CWNCRZRDPQXMII03LRIMLO4GURZ2K5U4VVP'
VERSION = '20200426' 


In [406]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            FOURSQUARE_CLIENT_ID, 
            FOURSQUARE_CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['BoroughName', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [480]:
LIMIT = 50
venues = getNearbyVenues(names=London['BoroughName'],
                                   latitudes=London['Latitude'],
                                   longitudes=London['Longitude']
                                  )

Barking and Dagenham
Barnet
Bexley
Brent
Bromley
Camden
Croydon
Ealing
Enfield
Greenwich
Hackney
Hammersmith and Fulham
Haringey
Harrow
Havering
Hillingdon
Hounslow
Islington
Kensington and Chelsea
Kingston upon Thames
Lambeth
Lewisham
Merton
Newham
Redbridge
Richmond upon Thames
Southwark
Sutton
Tower Hamlets
Waltham Forest
Wandsworth
Westminster


We now have our dataset. The next section, methodology, explores how we fit this data to generate recommendations for our client. 

# Methodology

To provide our location recommendations, we want to conduct a cluster analysis of the venues in each borough. 

First, we use one-hot encoding to display the distribution of venues across the neighbourhoods.

In [489]:
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")
borough = venues['BoroughName']
onehot.insert(0, 'BoroughName', borough)
onehot.head()


,BoroughName,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bistro,Boarding House,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Chaat Place,Champagne Bar,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,English Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Food Court,French Restaurant,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Mexican Restaurant,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Okonomiyaki Restaurant,Optical Shop,Organic Grocery,Outdoor Sculpture,Outlet Mall,Outlet Store,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pharmacy,Pizza Place,Platform,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Public Art,Rafting,Ramen Restaurant,Record Shop,Recording Studio,Restaurant,Roof Deck,Salad Place,Sandwich Place,Scenic Lookout,Shopping Mall,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Train Station,Turkish Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio
0,Barking and Dagenham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Barking and Dagenham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Barking and Dagenham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Barking and Dagenham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

We then group this dataframe by Boroughname, which provides a table of how these venues are distributed in each borough, and we then subset it to focus on Indian restaurants.

In [509]:
grouped = onehot.groupby('BoroughName').mean().reset_index()
grouped.head()
grouped2 = grouped[['BoroughName','African Restaurant', 'American Restaurant', 'Argentinian Restaurant', 'Asian Restaurant', 'Brazilian Restaurant', 'Caribbean Restaurant', 'Chinese Restaurant', 'Dim Sum Restaurant', 'Dumpling Restaurant', 'Falafel Restaurant', 'Fast Food Restaurant', 'Indian Restaurant', 'Indian Chinese Restaurant', 'Italian Restaurant', 'Japanese Restaurant', 'Korean Restaurant', 'Latin American Restaurant', 'Malay Restaurant', 'Mediterranean Restaurant', 'Mexican Restaurant', 'Modern European Restaurant', 'New American Restaurant', 'Okonomiyaki Restaurant', 'Persian Restaurant', 'Polish Restaurant', 'Portuguese Restaurant', 'Spanish Restaurant', 'Steakhouse', 'Sushi Restaurant', 'Tapas Restaurant', 'Thai Restaurant', 'Turkish Restaurant', 'Vietnamese Restaurant']]


In [510]:
num_top_venues = 5

for hood in grouped2['BoroughName']:
    print("----"+hood+"----")
    temp = grouped2[grouped2['BoroughName'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Barking and Dagenham----
                venue  freq
0  African Restaurant   0.0
1    Malay Restaurant   0.0
2  Turkish Restaurant   0.0
3     Thai Restaurant   0.0
4    Tapas Restaurant   0.0


----Barnet----
                venue  freq
0  African Restaurant   0.0
1    Malay Restaurant   0.0
2  Turkish Restaurant   0.0
3     Thai Restaurant   0.0
4    Tapas Restaurant   0.0


----Bexley----
                   venue  freq
0   Fast Food Restaurant  0.07
1     Italian Restaurant  0.07
2  Portuguese Restaurant  0.03
3    American Restaurant  0.03
4     Chinese Restaurant  0.03


----Brent----
                       venue  freq
0        American Restaurant  0.04
1  Latin American Restaurant  0.02
2           Asian Restaurant  0.02
3       Brazilian Restaurant  0.02
4          Indian Restaurant  0.02


----Bromley----
                   venue  freq
0   Fast Food Restaurant  0.03
1       Asian Restaurant  0.03
2       Sushi Restaurant  0.03
3  Portuguese Restaurant  0.03
4     African Re

In [511]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# Labelling columns as 1st, 2nd and so on
columns = ['BoroughName']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind])) # for 1st, 2nd, 3rd
    except:
        columns.append('{}th Most Common Venue'.format(ind+1)) ### for 4th, 5th,...

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns) ## assign column names we just created to a new dataframe
neighborhoods_venues_sorted['BoroughName'] = grouped2['BoroughName']## add neighborhoods column

for ind in np.arange(grouped2.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped2.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()
neighborhoods_venues_sorted.shape

(32, 11)

Apply the machine learning algorhythm to cluster the neighbourhoods based on the restaurants. 

In [525]:
k = 7

grouped_clustering = grouped2.drop('BoroughName', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:38]



array([3, 3, 0, 3, 3, 3, 1, 3, 3, 3, 3, 0, 5, 4, 3, 0, 2, 3, 3, 0, 6, 0,
       5, 3, 5, 0, 1, 0, 0, 3, 1, 3], dtype=int32)

In [526]:
London_merged = London
London_merged['Cluster Labels'] = kmeans.labels_
London_merged = London_merged.join(neighborhoods_venues_sorted.set_index('BoroughName'), on='BoroughName')
London_merged.head()

,BoroughName,Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barking and Dagenham,194352,51.5607,0.1557,3,Vietnamese Restaurant,Korean Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Brazilian Restaurant,Caribbean Restaurant,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant
1,Barnet,369088,51.6252,-0.1517,3,Vietnamese Restaurant,Korean Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Brazilian Restaurant,Caribbean Restaurant,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant
2,Bexley,236687,51.4549,0.1505,0,Italian Restaurant,Fast Food Restaurant,Chinese Restaurant,American Restaurant,Portuguese Restaurant,Dumpling Restaurant,Indian Chinese Restaurant,Indian Restaurant,Falafel Restaurant,Vietnamese Restaurant
3,Brent,317264,51.5588,-0.2817,3,American Restaurant,Latin American Restaurant,Asian Restaurant,Brazilian Restaurant,Indian Restaurant,Korean Restaurant,Argentinian Restaurant,Caribbean Restaurant,Chinese Restaurant,Dim Sum Restaurant
4,Bromley,317899,51.4039,0.0198,3,Sushi Restaurant,Asian Restaurant,Fast Food Restaurant,Portuguese Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Indian Chinese Restaurant,Indian Restaurant,Falafel Restaurant,Dumpling Restaurant


## Results Part 1: Exploring the Clusters

In [527]:
London_merged.loc[London_merged['Cluster Labels'] == 0, London_merged.columns[[1] + list(range(5, London_merged.shape[1]))]]

,Population,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,236687,Italian Restaurant,Fast Food Restaurant,Chinese Restaurant,American Restaurant,Portuguese Restaurant,Dumpling Restaurant,Indian Chinese Restaurant,Indian Restaurant,Falafel Restaurant,Vietnamese Restaurant
11,178685,Italian Restaurant,Indian Restaurant,Japanese Restaurant,Vietnamese Restaurant,Polish Restaurant,Chinese Restaurant,Persian Restaurant,Indian Chinese Restaurant,Portuguese Restaurant,Sushi Restaurant
15,286806,Italian Restaurant,Indian Restaurant,Asian Restaurant,Brazilian Restaurant,Mexican Restaurant,Portuguese Restaurant,Fast Food Restaurant,Dumpling Restaurant,Indian Chinese Restaurant,Falafel Restaurant
19,166793,Italian Restaurant,Thai Restaurant,Sushi Restaurant,Latin American Restaurant,Portuguese Restaurant,Mexican Restaurant,Japanese Restaurant,Asian Restaurant,Brazilian Restaurant,Argentinian Restaurant
21,286180,Italian Restaurant,Japanese Restaurant,Fast Food Restaurant,Portuguese Restaurant,Dim Sum Restaurant,Indian Chinese Restaurant,Indian Restaurant,Falafel Restaurant,Dumpling Restaurant,Vietnamese Restaurant
25,191365,Italian Restaurant,Indian Restaurant,Portuguese Restaurant,Mediterranean Restaurant,Fast Food Restaurant,Japanese Restaurant,Vietnamese Restaurant,Steakhouse,Brazilian Restaurant,Caribbean Restaurant
27,195914,Italian Restaurant,Fast Food Restaurant,Portuguese Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Indian Chinese Restaurant,Indian Restaurant,Falafel Restaurant,Dumpling Restaurant,Chinese Restaurant
28,272890,Italian Restaurant,Asian Restaurant,Chinese Restaurant,Vietnamese Restaurant,Korean Restaurant,American Restaurant,Argentinian Restaurant,Brazilian Restaurant,Caribbean Restaurant,Dim Sum Restaurant


In [528]:
London_merged.loc[London_merged['Cluster Labels'] == 1, London_merged.columns[[1] + list(range(5, London_merged.shape[1]))]]

,Population,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,372752,Portuguese Restaurant,Asian Restaurant,Caribbean Restaurant,Korean Restaurant,Malay Restaurant,Mediterranean Restaurant,Indian Restaurant,Italian Restaurant,Spanish Restaurant,Sushi Restaurant
26,298464,Indian Restaurant,Vietnamese Restaurant,Thai Restaurant,Tapas Restaurant,Steakhouse,Argentinian Restaurant,Asian Restaurant,Italian Restaurant,Mexican Restaurant,Dumpling Restaurant
30,310516,Indian Restaurant,Asian Restaurant,Thai Restaurant,Portuguese Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Indian Chinese Restaurant,Fast Food Restaurant,Falafel Restaurant,Dumpling Restaurant


In [529]:
London_merged.loc[London_merged['Cluster Labels'] == 2, London_merged.columns[[1] + list(range(5, London_merged.shape[1]))]]

,Population,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,262407,Chinese Restaurant,Vietnamese Restaurant,Korean Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Brazilian Restaurant,Caribbean Restaurant,Dim Sum Restaurant,Dumpling Restaurant


In [530]:
London_merged.loc[London_merged['Cluster Labels'] == 3, London_merged.columns[[1] + list(range(5, London_merged.shape[1]))]]

,Population,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,194352,Vietnamese Restaurant,Korean Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Brazilian Restaurant,Caribbean Restaurant,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant
1,369088,Vietnamese Restaurant,Korean Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Brazilian Restaurant,Caribbean Restaurant,Chinese Restaurant,Dim Sum Restaurant,Dumpling Restaurant
3,317264,American Restaurant,Latin American Restaurant,Asian Restaurant,Brazilian Restaurant,Indian Restaurant,Korean Restaurant,Argentinian Restaurant,Caribbean Restaurant,Chinese Restaurant,Dim Sum Restaurant
4,317899,Sushi Restaurant,Asian Restaurant,Fast Food Restaurant,Portuguese Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Indian Chinese Restaurant,Indian Restaurant,Falafel Restaurant,Dumpling Restaurant
5,229719,Turkish Restaurant,Malay Restaurant,Sushi Restaurant,Modern European Restaurant,Falafel Restaurant,Italian Restaurant,Vietnamese Restaurant,Dumpling Restaurant,Indian Restaurant,Fast Food Restaurant
7,342494,Vietnamese Restaurant,Italian Restaurant,Portuguese Restaurant,Fast Food Restaurant,Caribbean Restaurant,Polish Restaurant,Persian Restaurant,Spanish Restaurant,Thai Restaurant,Asian Restaurant
8,320524,Turkish Restaurant,Italian Restaurant,Indian Restaurant,Fast Food Restaurant,Portuguese Restaurant,Dim Sum Restaurant,Indian Chinese Restaurant,Falafel Restaurant,Dumpling Restaurant,Vietnamese Restaurant
9,264008,Fast Food Restaurant,Vietnamese Restaurant,Steakhouse,Asian Restaurant,African Restaurant,Thai Restaurant,Sushi Restaurant,Spanish Restaurant,American Restaurant,Argentinian Restaurant
10,257379,Vietnamese Restaurant,Modern European Restaurant,Italian Restaurant,Turkish Restaurant,Asian Restaurant,Brazilian Restaurant,Caribbean Restaurant,Chinese Restaurant,Japanese Restaurant,Dumpling Restaurant
14,242080,Fast Food Restaurant,Vietnamese Restaurant,Korean Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Brazilian Restaurant,Caribbean Restaurant,Chinese Restaurant,Dim Sum Restaurant


In [531]:
London_merged.loc[London_merged['Cluster Labels'] == 4, London_merged.columns[[1] + list(range(5, London_merged.shape[1]))]]

,Population,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,243372,Indian Restaurant,Vietnamese Restaurant,Korean Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Brazilian Restaurant,Caribbean Restaurant,Chinese Restaurant,Dim Sum Restaurant


In [532]:
London_merged.loc[London_merged['Cluster Labels'] == 5, London_merged.columns[[1] + list(range(5, London_merged.shape[1]))]]

,Population,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,263386,Fast Food Restaurant,Italian Restaurant,Turkish Restaurant,Mediterranean Restaurant,Indian Restaurant,Portuguese Restaurant,Dumpling Restaurant,Indian Chinese Restaurant,Falafel Restaurant,Vietnamese Restaurant
22,203223,Fast Food Restaurant,Indian Restaurant,Italian Restaurant,Vietnamese Restaurant,Korean Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Brazilian Restaurant,Caribbean Restaurant
24,288272,Fast Food Restaurant,Turkish Restaurant,Indian Restaurant,Portuguese Restaurant,Dim Sum Restaurant,Italian Restaurant,Indian Chinese Restaurant,Falafel Restaurant,Dumpling Restaurant,Vietnamese Restaurant


In [533]:
London_merged.loc[London_merged['Cluster Labels'] == 6, London_merged.columns[[1] + list(range(5, London_merged.shape[1]))]]

,Population,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,314242,Caribbean Restaurant,Indian Restaurant,Tapas Restaurant,African Restaurant,Okonomiyaki Restaurant,Dumpling Restaurant,Mexican Restaurant,Dim Sum Restaurant,New American Restaurant,Japanese Restaurant


## Results Part Two: Visualising the Clusters

In [534]:
map_clusters = folium.Map(location=[LON_LATITUDE,LON_LONGITUDE], zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i+x+(i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(London_merged['Latitude'], London_merged['Longitude'], London_merged['BoroughName'], London_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters